In [63]:
# pytorch
import torch
from torch import nn
# import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch import optim
from torch.autograd import Variable

# Helper libraries
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import random
from sklearn.decomposition import IncrementalPCA
#sys
import os
from collections import OrderedDict
import glob
import math

In [64]:
def same_seeds(seed):
    # python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# 为了结果可复现
Seed = 42
same_seeds(Seed)

In [65]:
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count() / 7)

In [66]:
# 加载数据
raw_data = pd.read_csv('/home/jsm/code/python/IoT-botnet/data/UNSW-NB15 - CSV Files/unsw15_train.csv')
temp = raw_data.loc[raw_data['attack_cat'] == 'Normal']
# temp = temp.drop(['attack_cat', 'label'], axis=1, inplace=False)
# temp.head()
# temp1 = temp.sample(1000*3, random_state=Seed)

In [67]:
temp_drop = temp.drop(['196', 'attack_cat', 'label'], axis=1, inplace=False)
temp_sameple = temp_drop.sample(1024*60, random_state=Seed)

In [68]:
temp_sameple.shape
tr_data = temp_sameple

In [69]:
# ipca = IncrementalPCA(n_components=128, batch_size=160)
# tr_data = ipca.fit_transform(temp_sameple.values)
# print(tr_data.shape)

In [70]:
class MyDataset(Dataset):
    def __init__(
        self,
        # batch_size,
        # num_workers,
        data
    ):
        # 在数据1维处增加1个维度 example: (64, 197) --> (64, 1, 197)
        # self.batch_size = batch_size
        # self.num_workers = num_workers
        self.data = data.unsqueeze(1)
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)

dataset = MyDataset(torch.from_numpy(tr_data.values).float())

In [71]:
#在处理好数据后定义
# InputLength = 64
workspace_dir = '.'

In [72]:
# 网络参数初始化
def weights_init(m):
    classname = m.__class__.__name__
    # 初始化网络层
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [73]:
# 生成器
class Generator(nn.Module):
    """
    Input shape: (N, 1, in_dim)
    Output shape: (N, 1, in_dim)
    """
    def __init__(self, in_dim, dim=32):
        super(Generator, self).__init__()
        self.in_dim = in_dim
        self.dim = dim
        self.inlayer = nn.Sequential(
            nn.Linear(self.in_dim, self.in_dim, bias=False),
            # tf 默认为0.3， torch 默认为0.01
            nn.LeakyReLU(negative_slope=0.2)
        )
        self.midlayer = nn.Sequential(
            # tf中一维卷积filter表示卷积核的个数，与torch中的out_channel相同
            nn.Conv1d(1, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=0.2),

            nn.Conv1d(32, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=0.2),

            nn.Conv1d(32, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=0.2)
        )
        self.outlayer = nn.Sequential(
            nn.Conv1d(32, 1, kernel_size=3, padding=1),
            nn.Tanh()
        )
        self.apply(weights_init)

    def forward(self, x):
        y = self.inlayer(x)
        y = self.midlayer(y)
        y = self.outlayer(y)
        return y

In [74]:
# 判别器
class Discriminator(nn.Module):
    """
    Input shape: (N, 1, in_dim)
    Output shape: (N, )
    """
    def __init__(self, in_dim, in_channel=1, batch=32):
        super(Discriminator, self).__init__()
        self.in_channel = in_channel
        self.in_dim = in_dim
        self.batch = batch
        # self.channel = channel
        self.inlayer = nn.Sequential(
            nn.Conv1d(in_channel, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=0.2),
            nn.MaxPool1d(kernel_size = 2) # shape: (N, 32, in_dim/2)
        )
        self.midlayer1 = nn.Sequential(
            nn.Conv1d(32, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=0.2),
            nn.MaxPool1d(kernel_size=2), # shape: (N, 32, in_dim/2/2)
            nn.Flatten(), # shape: (N, 32*in_dim/2/2)
        )
        self.temp_dim = 32 * math.floor(math.floor(self.in_dim / 2) / 2)
        self.midlayer2 = nn.Sequential(
            nn.Linear(self.temp_dim, 64),
            nn.Dropout(0.4),
            nn.LeakyReLU(negative_slope=0.2)
        )
        self.outlayer = nn.Linear(64, 1)
    
    def forward(self, x):
        y = self.inlayer(x)
        y = self.midlayer1(y)
        y = self.midlayer2(y)
        y = self.outlayer(y)
        y = y.view(-1)
        return y        

In [75]:
class WGAN(LightningModule):
    def __init__(
        self,
        in_dim: int = 16,
        in_channels: int = 1,
        lr: float = 1e-4,
        n_critic: int = 5,
        clip_value: float = 0.1,
        batch_size: int = BATCH_SIZE,
        **kwargs
    ):
        super().__init__()
        self.save_hyperparameters()

        # networks
        self.generator = Generator(in_dim=self.hparams.in_dim)
        self.discriminator = Discriminator(in_dim=self.hparams.in_dim, in_channel=self.hparams.in_channels)
        self.validation_z = torch.randn(10, self.hparams.in_channels, self.hparams.in_dim)

    def forward(self, z):
        return self.generator(z)

    def adversarial_loss(self, y_hat, y):
        return -torch.mean(self.discriminator(y)) + torch.mean(self.discriminator(y_hat))
    
    def training_step(self, batch, batch_idx, optimizer_idx):
        # print(batch.size())
        # data, _ = batch
        # print(data.size())
        # sample noise
        data = batch
        z = torch.randn(self.hparams.batch_size, self.hparams.in_channels, self.hparams.in_dim)
        z = z.type_as(data)

        # train generator
        if optimizer_idx == 0 and (batch_idx % self.hparams.n_critic == 0 and batch_idx != 0):
            
            # generate data
            self.generated_data = self(z)
            
            # log sampled data
            # sample_data = self.generated_data[:10]
            # self.logger.experiment.add_scalar("generated_data", sample_data[0], 0)

            # generator of WGAN loss
            g_loss = -torch.mean(self.discriminator(self(z)))
            tqdm_dict = {"g_loss": g_loss}
            output = OrderedDict({"loss": g_loss, "progress_bar": tqdm_dict, "log": tqdm_dict})
            return output

        if optimizer_idx == 1:
            # discriminator of WGAN loss
            d_loss = -torch.mean(self.discriminator(data)) + torch.mean(self.discriminator(self(z)))
            
            # Clip weights of discriminator
            for p in self.discriminator.parameters():
                p.data.clamp_(-self.hparams.clip_value, self.hparams.clip_value)

            tqdm_dict = {"d_loss": d_loss}
            output = OrderedDict({"loss": d_loss, "progress_bar": tqdm_dict, "log": tqdm_dict})
            return output

    def configure_optimizers(self):
        lr = self.hparams.lr

        opt_g = torch.optim.RMSprop(self.generator.parameters(), lr=lr)
        opt_d = torch.optim.RMSprop(self.discriminator.parameters(), lr=lr)
        
        return [opt_g, opt_d], []

    # def on_epoch_end(self):
    #     z = self.validation_z.type_as(self.generator.model[0].weight)

    #     # log sampled data
    #     sample_data = self(z)
    #     # self.logger.experiment.add_scalar("generated_data", sample_data[0], self.current_epoch)
    #     self.log("generated_data", sample_data, logger=True)

In [76]:
tr_dataloder = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
in_dim = tr_data.shape[-1]
wgan = WGAN(in_dim)

In [77]:
trainer = Trainer(
    gpus = AVAIL_GPUS,
    max_epochs=10,
    progress_bar_refresh_rate = 20
)
trainer.fit(wgan, tr_dataloder)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 44.8 K
1 | discriminator | Discriminator | 103 K 
------------------------------------------------
148 K     Trainable params
0         Non-trainable params
148 K     Total params
0.594     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

In [10]:
# Trainnig hyperparmeters
batch_size = 128
in_dim = tr_data.shape[-1]
z_dim = in_dim
z_sample = Variable(torch.randn(batch_size, 1, z_dim)).cuda()
lr = 1e-4

n_epoch = 10000
n_critic = 5
# 待改
clip_value = 0.1

ckpt_dir = os.path.join(workspace_dir, 'checkpoints')
os.makedirs(ckpt_dir, exist_ok=True)

# Model
G = Generator(in_dim).cuda()
D = Discriminator(in_dim, 1).cuda()
G.train()
D.train()

# Loss
# criterion = nn.BCELoss()

# Optimizer
opt_D = torch.optim.RMSprop(D.parameters(), lr=lr)
opt_G = torch.optim.RMSprop(D.parameters(), lr=lr)

#DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=1)

In [11]:
steps = 0
for e, epoch in enumerate(range(n_epoch)):
    # progress_bar = qqdm(dataloader)
    for i, data in enumerate(dataloader):
        mid_data = data
        # print(mid_data.size())
        mid_data = mid_data.cuda()
        bs = mid_data.size(0)

        # ============================================
        #  Train D
        # ============================================
        z = Variable(torch.randn(bs, 1, z_dim)).cuda()
        r_data = Variable(mid_data).cuda()
        f_data = G(z)
        # print(f_data)
        # """ Medium: Use WGAN Loss. """
        # Label
        r_label = torch.ones((bs)).cuda()
        f_label = torch.zeros((bs)).cuda()

        # Model forwarding
        r_logit = D(r_data.detach())
        f_logit = D(f_data.detach())
        # # Compute the loss for the discriminator
        # r_loss = criterion(r_logit, r_label) 
        # f_loss = criterion(f_logit, f_label)
        # # loss_D = (r_loss + f_loss) / 2

        # WGAN Loss
        loss_D = -torch.mean(D(r_data)) + torch.mean(D(f_data))

        # Model backwarding
        D.zero_grad()
        loss_D.backward()

        # Updata the discriminator
        opt_D.step()

        """ Medium: Clip weights of discriminator. """
        for p in D.parameters():
            p.data.clamp_(-clip_value, clip_value)
        
        # ============================================
        #  Train G
        # ============================================
        if steps % n_critic == 0:
            # Generate some fake data
            z = Variable(torch.randn(bs, 1, z_dim)).cuda()
            f_data = G(z)

            # Model forearding
            f_logit = D(f_data)

            # WGAN Loss
            loss_G = -torch.mean(D(f_data))

            # Model backwarding
            G.zero_grad()
            loss_G.backward()

            # Updata the generator
            opt_G.step()

            
        steps += 1
        '''
        Set the info of the progress bar.
        Note that the value of the GAN loss is not directly related to the quality of the generated images.
        '''
        # progress_bar.set_infos({
        #     'Loss_D': round(loss_D.item(), 4),
        #     'Loss_G': round(loss_G.item(), 4),
        #     'Epoch': e+1,
        #     'Step': steps,
        # })
        
    print('Loss_D {}, Loss_G {}, Epoch {}, Step {}'.format(
        round(loss_D.item(), 4),
        round(loss_G.item(), 4),
        e+1,
        steps, ))

    G.eval()
    f_data_sample = G(z_sample).data
    G.train()

    if (e+1) % 5 == 0 or e == 0:
        # Save the checkpoints
        torch.save(G.state_dict(), os.path.join(ckpt_dir, 'G.pth'))
        torch.save(D.state_dict(), os.path.join(ckpt_dir, 'D.pth'))

/home/jsm/miniconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Loss_D -75388956966912.0, Loss_G -1.3917, Epoch 1, Step 24
Loss_D -213484604227584.0, Loss_G -4.1533, Epoch 2, Step 48
Loss_D -460979309641728.0, Loss_G -7.812, Epoch 3, Step 72
Loss_D -805020954001408.0, Loss_G -12.3147, Epoch 4, Step 96
Loss_D -948578692366336.0, Loss_G -16.1503, Epoch 5, Step 120
Loss_D -1729318652739584.0, Loss_G -20.9663, Epoch 6, Step 144
Loss_D -1943167221891072.0, Loss_G -25.5731, Epoch 7, Step 168
Loss_D -2691044508434432.0, Loss_G -30.9085, Epoch 8, Step 192
Loss_D -2225887202246656.0, Loss_G -34.7276, Epoch 9, Step 216
Loss_D -3411426219655168.0, Loss_G -39.5049, Epoch 10, Step 240
Loss_D -3551466849566720.0, Loss_G -44.4038, Epoch 11, Step 264
Loss_D -3654698267574272.0, Loss_G -48.4342, Epoch 12, Step 288
Loss_D -4166095996977152.0, Loss_G -51.9833, Epoch 13, Step 312
Loss_D -4354983289946112.0, Loss_G -55.8829, Epoch 14, Step 336
Loss_D -4648864011255808.0, Loss_G -58.7874, Epoch 15, Step 360
Loss_D -4664641036746752.0, Loss_G -60.1054, Epoch 16, Step 384